In [1]:
import pandas as pd
import numpy 
import os
import git
import pickle
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', -1)
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import warnings
warnings.filterwarnings('ignore')
from matplotlib import pyplot
import numpy as np
import re
from collections import Counter
# import gensim
# from gensim import corpora
import os
import networkx as nx
import ast
import xgboost
from xgboost import XGBClassifier
from tqdm import tqdm

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.utils import shuffle

In [3]:
## Read the labelled files and the poly_user
def get_git_root(path):
    git_repo = git.Repo(path, search_parent_directories=True)
    git_root = git_repo.git.rev_parse("--show-toplevel")
    return git_root

In [4]:
top_dir = os.path.join(get_git_root(os.getcwd()))
input_dir = os.path.join(get_git_root(os.getcwd()),"input")
embeddings_dir  = os.path.join(get_git_root(os.getcwd()),"input","embeddings")
annotatted_dir = os.path.join(input_dir,"annotated_data")
classifier_dir = os.path.join(get_git_root(os.getcwd()),"models","classifier")
model_dir = os.path.join(get_git_root(os.getcwd()),"models")
poly_dir = os.path.join(model_dir,"poly_users")

In [5]:
poly_users = pickle.load(open(os.path.join(poly_dir,"poly_users.pkl"),"rb"))
regular_users = pickle.load(open(os.path.join(poly_dir,"regular_users.pkl"),"rb"))
mono_users = pickle.load(open(os.path.join(poly_dir,"mono_users.pkl"),"rb"))

In [ ]:
juul_data = pd.read_csv(os.path.join(input_dir,"juul_data.csv"),lineterminator="\n")

In [ ]:
### building the network
def get_graph(df)-> nx.DiGraph():
    G = nx.DiGraph()
    users = list(df.userID.unique())
    for user in tqdm(users):
        following_A = set(ast.literal_eval((df.loc[df.userID == user].head(1)["following"].values)[0]))
        user_set = set([node for node in users if node != user])
        users_list = user_set.intersection(following_A)
        for user_following in list(users_list):
            G.add_edge(user, user_following)
    return G

In [8]:
following_new = pd.read_csv(os.path.join(input_dir,"following_new.csv"))

In [32]:
len(following_new)

36587

In [30]:
G = get_graph(following_new)

100%|██████████| 36587/36587 [05:14<00:00, 116.42it/s]


In [31]:
len(G.nodes())

25697

In [36]:
color_map = list()
for node in G.nodes():
    if int(node) in poly_users:
        color_map.append('green')
    else:
        color_map.append('blue')

In [37]:
len(G.nodes())

25697

In [40]:
# nx.draw(G,node_color=color_map)

In [41]:
## dump the network graph
nx.write_gpickle(G, os.path.join(model_dir,"network","following_network.gpickle"))

In [43]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 25697
Number of edges: 63835
Average in degree:   2.4841
Average out degree:   2.4841
